In [162]:
import pandas as pd
import numpy as np
import regex
from transformers import BertTokenizer


In [153]:
dataframes = []
for i in range(9):
    dataframes.append(pd.read_csv(f"src/trolls/IRAhandle_tweets_{i+1}.csv"))
trolldata = pd.concat(dataframes)

genuine = pd.read_csv("src/genuine/Political_tweets.csv")


C:\Users\Jayden\AppData\Local\Temp\ipykernel_23528\437199962.py:6: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  genuine = pd.read_csv("src/genuine/Political_tweets.csv")


***Data Pre-Processing***

**Removing Non-English entries**

In [154]:
#Removing non-english entries
troll = trolldata[trolldata["language"] == "English"]
# genuine.iloc[:100_000]
troll

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll
1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll
2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll
3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll
4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36991,3.093874e+09,_YOUR_LIFESTYLE,Like me. Share me. Follow me http://t.co/VvhUO...,Unknown,English,10/8/2015 15:01,10/8/2015 15:01,59,13,458,NaN,Russian,0,0,NonEnglish
37053,3.093874e+09,_YOUR_LIFESTYLE,.the main thing is to was love https://t.co/oq...,Unknown,English,11/4/2015 8:24,11/4/2015 8:25,68,23,518,NaN,Russian,0,0,NonEnglish
37337,3.093874e+09,_YOUR_LIFESTYLE,Amazing! Found my script: Download ReBuild - W...,United States,English,6/25/2016 11:27,6/25/2016 11:28,40,40,817,NaN,Russian,0,0,NonEnglish
37410,3.093874e+09,_YOUR_LIFESTYLE,Present Perfect http://t.co/vNu3i2Xh4M,Unknown,English,7/31/2015 18:35,7/31/2015 18:36,59,4,305,NaN,Russian,0,0,NonEnglish


**Finding any instances of known troll authors in genuine dataset**

In [155]:
#Getting list of troll authors
trollauthors = []
for author in troll['author'].unique():
    trollauthors.append(author)
trollauthors

#Getting list of genuine authors
genuineauthors = genuine['user_name'].apply(lambda x: x.upper() if isinstance(x, str) else x)

#Searching for identical names

matches = []
for rname,tname in zip(genuineauthors,trollauthors):
    if rname == tname:
        matches.append(rname)

print(f"Found {len(matches)} matches: {matches}")



Found 0 matches: []


In [156]:
genuine

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,CPA Jim 2021,NaN,Grandparents enslaved by CCP since 1949 were b...,2011-03-17 10:17:36,711.0,146.0,1160.0,False,2021-07-30 23:57:05,US says seizes tanker used to evade North Kore...,"['politics', 'feedly']",Twitter Web App,False
1,Joint Minds,NaN,Joint Minds Podcast,2021-01-23 23:57:27,3.0,3.0,1.0,False,2021-07-30 23:53:21,Is America the greatest country? Part 1 drops ...,"['america', 'jointminds', 'politics', 'democra...",Twitter for iPhone,False
2,Sociafy,USA,Sociafy is an influencer-based content discove...,2011-09-30 05:59:06,36219.0,181.0,930.0,False,2021-07-30 23:53:03,O’Reilly predicts THIS is What DOOMS Biden’s r...,"['politics', 'videos']",Buffer,False
3,🟣 Discussions.app,United States,Home of Crypto Communities. $ATMOS \n\nhttps:/...,2017-02-09 23:15:24,2426.0,1104.0,8173.0,False,2021-07-30 23:51:00,Corruption in Politics\n#blog #politics \nhttp...,"['blog', 'politics']",TweetDeck,False
4,Ground News,"Kitchener, ON",The world's first news source comparison platf...,2018-02-08 22:00:16,13763.0,1224.0,2033.0,False,2021-07-30 23:50:53,@kylegriffin1 Are you getting all the info on ...,"['EvictionMoratorium', 'Congress', 'NancyPelos...",GroundNews,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
238641,ViralNews,NaN,#OSINT #AI trending news aggregator\nMost up-t...,2009-04-17 01:37:33,125.0,686.0,1010.0,False,2022-08-15 21:58:13,Morata double fires Atletico to opening win at...,"['breaking', 'breakingnews', 'politics', 'usa'...",IFTTT,False
238642,Angelo Tsanatelis,Greece,Author/ Earth citizen. #FB #Grimdark https://...,2011-04-10 22:37:11,2157.0,2388.0,373.0,False,2022-08-15 21:58:09,Read chapter 31 at Scribble Hub! \n\n#Novel #h...,"['Novel', 'highfantasy', 'fantasy', 'Grimdark'...",Twitter Web App,False
238643,Politic Talks,NaN,PoliticTalks welcomes your #political conversa...,2018-06-20 19:24:07,380.0,45.0,9.0,False,2022-08-15 21:58:02,Biden administration to launch nationwide tour...,"['Politics', 'Political']",dlvr.it,False
238644,Demonic Machines (the pedal company),"San Diego, CA",Profile run by Lucky (the monster) (ze/hir/hir...,2021-06-04 23:31:47,69.0,739.0,665.0,False,2022-08-15 21:57:45,"if you have a #demonicmachines pedal, you shou...","['demonicmachines', 'politics', 'thismachineki...",Instagram,False


**Removal of unique metadata**<br>
We can't have any metadata that is strictly limited to one dataset. That will instantly cause bias.

I also removed information about the authors of the tweet.

In addition, column names have been changed to fit to the troll dataset

In [157]:
troll.drop(columns=["language","harvested_date","publish_date","external_author_id","following","account_type","new_june_2018","region","post_type","updates","account_category"],inplace=True)
troll.drop(columns=["author"],inplace=True)
genuine.drop(columns=['user_name','user_location','user_description','user_created','user_friends','user_favourites','user_verified','date','hashtags','source'],inplace=True)

namechanges = {
"user_followers":"followers",
"is_retweet":"retweet",
"text":"content",

}

genuine.rename(columns=namechanges,inplace=True)



C:\Users\Jayden\AppData\Local\Temp\ipykernel_23528\1357675685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  troll.drop(columns=["language","harvested_date","publish_date","external_author_id","following","account_type","new_june_2018","region","post_type","updates","account_category"],inplace=True)
C:\Users\Jayden\AppData\Local\Temp\ipykernel_23528\1357675685.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  troll.drop(columns=["author"],inplace=True)


**Add labels to datasets and concatenate**

In [158]:
genuine['retweet'].unique()

array([False, nan], dtype=object)

In [159]:
genlabels = np.zeros(shape=(len(genuine),1))


trolabels = np.ones(shape=(len(troll),1))

genuine['troll'] = genlabels
troll['troll'] = trolabels
genuine['retweet'] = genuine['retweet'].apply(lambda x: 0.0 if x == False else 1.0)

bigdataset = pd.concat([genuine.reset_index(drop=True), troll.reset_index(drop=True)], axis=0)
bigdataset


C:\Users\Jayden\AppData\Local\Temp\ipykernel_23528\1789973570.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  troll['troll'] = trolabels


,followers,content,retweet,troll
0,711.0,US says seizes tanker used to evade North Kore...,0.0,0.0
1,3.0,Is America the greatest country? Part 1 drops ...,0.0,0.0
2,36219.0,O’Reilly predicts THIS is What DOOMS Biden’s r...,0.0,0.0
3,2426.0,Corruption in Politics\n#blog #politics \nhttp...,0.0,0.0
4,13763.0,@kylegriffin1 Are you getting all the info on ...,0.0,0.0
...,...,...,...,...
2128958,13.0,Like me. Share me. Follow me http://t.co/VvhUO...,0.0,1.0
2128959,23.0,.the main thing is to was love https://t.co/oq...,0.0,1.0
2128960,40.0,Amazing! Found my script: Download ReBuild - W...,0.0,1.0
2128961,4.0,Present Perfect http://t.co/vNu3i2Xh4M,0.0,1.0


**Normalization**
The only column that needs normalization at this stage is followers. So I min-max normalized it

In [168]:

def minmax(iter:pd.Series):
    mx = iter.max()
    min = iter.min()
    normalized = []
    for val in iter:
        normalized.append( (val - min)/mx-min)
    return pd.Series(normalized)


bigdataset['followers'] = minmax(bigdataset['followers'])

bigdataset.reset_index(inplace=True)
bigdataset

,level_0,index,followers,content,retweet,troll
0,0,0,5.023806e-05,US says seizes tanker used to evade North Kore...,0.0,0.0
1,1,1,2.119749e-07,Is America the greatest country? Part 1 drops ...,0.0,0.0
2,2,2,2.559174e-03,O’Reilly predicts THIS is What DOOMS Biden’s r...,0.0,0.0
3,3,3,1.714171e-04,Corruption in Politics\n#blog #politics \nhttp...,0.0,0.0
4,4,4,9.724704e-04,@kylegriffin1 Are you getting all the info on ...,0.0,0.0
...,...,...,...,...,...,...
2367604,2367604,2128958,1.833159e-03,Like me. Share me. Follow me http://t.co/VvhUO...,0.0,1.0
2367605,2367605,2128959,1.833159e-03,.the main thing is to was love https://t.co/oq...,0.0,1.0
2367606,2367606,2128960,1.833159e-03,Amazing! Found my script: Download ReBuild - W...,0.0,1.0
2367607,2367607,2128961,1.833159e-03,Present Perfect http://t.co/vNu3i2Xh4M,0.0,1.0


**Preparation for Transformer input**
I didn't change the capitalization because it contains potentially useful information. BERT cased will capture this context

In [169]:
bigdataset['content'][0]

'US says seizes tanker used to evade North Korea sanctions https://t.co/ABaaAcIUnr #politics #feedly'

In [185]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

text = bigdataset['content'][0]
encoded_input = tokenizer.encode(text, add_special_tokens=True)



len(encoded_input)

# decoded_text = tokenizer.decode(encoded_input)
# decoded_text

31